In [43]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, mean_absolute_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

np.set_printoptions(precision=4)

In [44]:
data = pd.read_csv('final_df.csv')

display(data)

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,...,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
0,1983,1,False,False,True,False,False,keke_rosberg,1,15,...,0,0,0,0,0,0,0,0,0,1
1,1983,1,False,False,True,False,False,prost,2,6,...,0,0,0,1,0,0,0,0,0,0
2,1983,1,False,False,True,False,False,tambay,3,4,...,0,0,0,0,0,0,0,0,0,0
3,1983,1,False,False,True,False,False,piquet,4,1,...,0,0,0,0,0,0,0,0,0,0
4,1983,1,False,False,True,False,False,warwick,5,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14268,2019,21,True,False,False,False,False,giovinazzi,16,16,...,0,0,0,0,0,0,0,0,0,0
14269,2019,21,True,False,False,False,False,raikkonen,17,13,...,0,0,0,0,0,0,0,0,0,0
14270,2019,21,True,False,False,False,False,russell,18,17,...,0,0,0,0,0,0,0,0,0,1
14271,2019,21,True,False,False,False,False,kubica,19,19,...,0,0,0,0,0,0,0,0,0,1


In [45]:
df = data.copy()

train = df[df.season <2019]
X_train = train.drop(['driver', 'podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [46]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

# Implement Model

In [47]:
model = GaussianProcessRegressor()
model.fit(X_train, y_train)

GaussianProcessRegressor()

# Evaluate Model

In [48]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == 2019]['round'].unique():

        test = df[(df.season == 2019) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        test = np.transpose(np.array(model.predict(X_test, return_std=True)))
        prediction_df = pd.DataFrame(test, columns = ['y_mean', 'y_std'])
        prediction_df['actual'] = y_test.reset_index(drop = True)

        prediction_df = prediction_df[prediction_df.y_std < 2]

        score += mean_absolute_error(prediction_df.actual, prediction_df.y_mean)

    model_score = score / df[df.season == 2019]['round'].unique().max()
    return model_score

In [49]:
model_score = score_classification(model)

comparison_dict['model'].append('gpr')
comparison_dict['score'].append(model_score)

display(comparison_dict)

{'model': ['gpr'], 'params': [], 'score': [9.315680148564038]}